In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError

# Download the dataset

data = pd.read_csv('icab1-mavros-imu-data.csv', header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,orientation.x,orientation.y,orientation.z,orientation.w,orientation_covariance_0,...,linear_acceleration.z,linear_acceleration_covariance_0,linear_acceleration_covariance_1,linear_acceleration_covariance_2,linear_acceleration_covariance_3,linear_acceleration_covariance_4,linear_acceleration_covariance_5,linear_acceleration_covariance_6,linear_acceleration_covariance_7,linear_acceleration_covariance_8
1,1616761551,11695,1616761549,749452672,fcu,-0.028768955,0.020188195,-0.986944485,-0.157179413,1,...,9.8851032,9.00E-08,0,0,0,9.00E-08,0,0,0,9.00E-08
2,1616761551,11696,1616761549,789452672,fcu,-0.028797252,0.020229031,-0.986932499,-0.157244226,1,...,9.87529655,9.00E-08,0,0,0,9.00E-08,0,0,0,9.00E-08
3,1616761551,11697,1616761549,830452672,fcu,-0.028795508,0.020233018,-0.986932104,-0.157246511,1,...,9.89490985,9.00E-08,0,0,0,9.00E-08,0,0,0,9.00E-08
4,1616761551,11698,1616761549,869452672,fcu,-0.028740509,0.020202095,-0.986932956,-0.157255203,1,...,9.87529655,9.00E-08,0,0,0,9.00E-08,0,0,0,9.00E-08


In [2]:
data.head(1)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,orientation.x,orientation.y,orientation.z,orientation.w,orientation_covariance_0,...,linear_acceleration.z,linear_acceleration_covariance_0,linear_acceleration_covariance_1,linear_acceleration_covariance_2,linear_acceleration_covariance_3,linear_acceleration_covariance_4,linear_acceleration_covariance_5,linear_acceleration_covariance_6,linear_acceleration_covariance_7,linear_acceleration_covariance_8


In [3]:
data.head(,1)

SyntaxError: invalid syntax (<ipython-input-3-1014a4910055>, line 1)

In [4]:
data.head(:1)

SyntaxError: invalid syntax (<ipython-input-4-4941790a23ae>, line 1)

In [5]:
data.head([:1])

SyntaxError: invalid syntax (<ipython-input-5-6b80e7cfc701>, line 1)

In [12]:
filter_data=data.loc[1:, [5,6,7,8,18,19,20,30,31,32]]

filter_data

,5,6,7,8,18,19,20,30,31,32
1,-0.028768955,0.020188195,-0.986944485,-0.157179413,0.001369239,0.003280692,-0.003239267,0.42168595,-0.2157463,9.8851032
2,-0.028797252,0.020229031,-0.986932499,-0.157244226,-0.000466197,0.002388213,-0.003292796,0.4118793,-0.2941995,9.87529655
3,-0.028795508,0.020233018,-0.986932104,-0.157246511,-0.001835544,0.002452841,-0.002718871,0.4118793,-0.30400615,9.89490985
4,-0.028740509,0.020202095,-0.986932956,-0.157255203,-0.00163346,0.004359301,-0.001778475,0.4118793,-0.2941995,9.87529655
5,-0.028744595,0.020114528,-0.986926413,-0.157306734,0.00020631,0.005040845,-0.002077841,0.40207265,-0.26477955,9.8851032
6,-0.028723184,0.020064384,-0.986931792,-0.157283303,0.001330691,0.005226405,-0.001748894,0.4314926,-0.2353596,9.87529655
7,-0.028807746,0.020062651,-0.986930298,-0.15727743,0.000236643,0.004022511,-0.000580793,0.4511059,-0.24516625,9.85568325
8,-0.028798075,0.020069094,-0.986930999,-0.157273984,-0.000161385,0.00266994,-0.000487654,0.4511059,-0.24516625,9.85568325
9,-0.028829678,0.020035988,-0.9869248,-0.157311311,-0.000644966,0.002654487,-0.001276786,0.44129925,-0.2353596,9.8654899
10,-0.028858665,0.020043491,-0.986926268,-0.157295825,-0.001084466,0.002944417,-0.00104423,0.38245935,-0.22555295,9.9047165


In [14]:
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
x_train_scaled = min_max_scaler.fit_transform(filter_data.copy())
x_train_scaled

array([[0.33544503, 0.92555214, 0.9975044 , ..., 0.38380282, 0.47860963,
        0.57357357],
       [0.33365194, 0.92768718, 0.99855347, ..., 0.38292254, 0.45721925,
        0.57057057],
       [0.33376245, 0.92789563, 0.99858805, ..., 0.38292254, 0.45454545,
        0.57657658],
       ...,
       [0.20067936, 0.96397916, 0.12444437, ..., 0.37147887, 0.43315508,
        0.56756757],
       [0.20123464, 0.96420596, 0.12436393, ..., 0.38028169, 0.42513369,
        0.57657658],
       [0.19869876, 0.96403061, 0.12468576, ..., 0.38644366, 0.42780749,
        0.56456456]])

In [17]:
# create a model by subclassing Model class in tensorflow
class AutoEncoder(Model):
  """
  Parameters
  ----------
  output_units: int
    Number of output units
  
  code_size: int
    Number of units in bottle neck
  """

  def __init__(self, output_units, code_size=8):
    super().__init__()
    self.encoder = Sequential([
      Dense(10, activation='relu'),
      Dropout(0.1),
      Dense(64, activation='relu'),
      Dropout(0.1),
      Dense(32, activation='relu'),
      Dropout(0.1),
      Dense(16, activation='relu')
    ])
    self.decoder = Sequential([
      Dense(16, activation='relu'),
      Dropout(0.1),
      Dense(32, activation='relu'),
      Dropout(0.1),
      Dense(64, activation='relu'),
      Dropout(0.1),
      Dense(10, activation='sigmoid')
    ])
  
  def call(self, inputs):
    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded
  
model = AutoEncoder(output_units=x_train_scaled.shape[1])
# configurations of model
model.compile(loss='msle', metrics=['mse'], optimizer='adam')

history = model.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=20,
    batch_size=512,
    
)

Epoch 1/20
1447/1447 [==============================] - 1s 358us/sample - loss: 0.0171 - mean_squared_error: 0.0361
Epoch 2/20
1447/1447 [==============================] - 0s 23us/sample - loss: 0.0168 - mean_squared_error: 0.0356
Epoch 3/20
1447/1447 [==============================] - 0s 26us/sample - loss: 0.0165 - mean_squared_error: 0.0350
Epoch 4/20
1447/1447 [==============================] - 0s 23us/sample - loss: 0.0162 - mean_squared_error: 0.0343
Epoch 5/20
1447/1447 [==============================] - 0s 21us/sample - loss: 0.0158 - mean_squared_error: 0.0336
Epoch 6/20
1447/1447 [==============================] - 0s 25us/sample - loss: 0.0154 - mean_squared_error: 0.0330
Epoch 7/20
1447/1447 [==============================] - 0s 23us/sample - loss: 0.0151 - mean_squared_error: 0.0327
Epoch 8/20
1447/1447 [==============================] - 0s 24us/sample - loss: 0.0150 - mean_squared_error: 0.0325
Epoch 9/20
1447/1447 [==============================] - 0s 25us/sample - loss: 

In [25]:
def find_threshold(model, x_train_scaled):
  # another method to find threshold
  reconstructions = model.predict(x_train_scaled)
  # provides losses of individual instances
  reconstruction_errors = tf.keras.losses.msle(reconstructions, x_train_scaled)

  threshold = np.percentile(reconstruction_errors, 95)
  return threshold

def get_predictions(model, x_test_scaled, threshold):
  predictions = model.predict(x_test_scaled)
  # provides losses of individual instances
  errors = tf.keras.losses.msle(predictions, x_test_scaled)
  # 0 = anomaly, 1 = normal
  anomaly_mask = pd.Series(errors) > threshold
  preds = anomaly_mask.map(lambda x: 0.0 if x == True else 1.0)
  return preds

threshold = find_threshold(model, x_train_scaled)
print(f"Threshold: {threshold}")
# Threshold: 0.01001314025746261
predictions = get_predictions(model, x_train_scaled, threshold)
accuracy_score(predictions, y_test)
# 0.944

Threshold: Tensor("add_12:0", shape=(1447,), dtype=float64)


TypeError: Tensor objects are only iterable when eager execution is enabled. To iterate over this tensor use tf.map_fn.

In [24]:
Threshold

NameError: name 'Threshold' is not defined

In [22]:
threshold

<tf.Tensor 'add_4:0' shape=(1447,) dtype=float64>